<a href="https://colab.research.google.com/github/ElinaNeu/Project-Week-1-Build-Your-Own-Game/blob/master/Project_EN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#spotipy installen
!pip install spotipy

In [15]:
#General installed
import re 
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import random
from google.colab import drive
import pandas as pd
import numpy as np
drive.mount('/content/drive')
import sys
sys.path.insert(1, '/content/drive/MyDrive/credentials/')
from config import *
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
#import top 100 from website
url = "https://www.billboard.com/charts/hot-100"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [17]:
#Billboard Hot 100, songlist and songtitles
#song titles of billboard HOT 100, via url
song_titles = []
for tag in soup.select("button > span.chart-element__information > span.chart-element__information__song.text--truncate.color--primary"):
    song_titles.append(tag.get_text())
#song artists of Billboard Hot 100, via url
song_artists = []
for i in soup.select("button > span.chart-element__information > span.chart-element__information__artist.text--truncate.color--secondary"):
    song_artists.append(i.get_text())
# Dataphrame of artists and Titles Hot 100
songlist = pd.DataFrame({'song_titles': song_titles, 'song_artists': song_artists})

In [18]:
#Function Billboard100 
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    return tracks

top_100 = get_playlist_tracks("spotify", "6UeSakyzhiEt4NB3UAd6NQ")
all_dicts = []

#Funktion Billboard100 audiofeautures
for i in range(len(top_100)):
  uri = top_100[i]["track"]["uri"] 
  all_dicts.append(list(sp.audio_features(uri)[0].values()))
len(all_dicts)

#PUT AUDIO FEATURES INTO A PANDAS DATAFRAME
df = pd.DataFrame(all_dicts, columns=['danceability', 'energy', 'key', 'loudness', 'mode', 
                                      'speechiness','acousticness','instrumentalness','liveness','valence',
                                      'tempo','type','id','uri','track_href',
                                      'analysis_url','duration_ms','time_signature'])

In [34]:
#Final Dataphrame Top100
final100 = pd.concat([songlist, df], axis = 1)

In [20]:
#Playlist Spotify
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

#Playlist Spotify Songtitles & artists
all_tracks = get_playlist_tracks("spotify", "6yPiKpy7evrwvZodByKvM9")
song_titles = [all_tracks[i]["track"]["name"] for i in range(len(all_tracks))]
song_artists = [all_tracks[i]["track"]["artists"][0]["name"] for i in range(len(all_tracks))]
spotify_data = pd.DataFrame(list(zip(song_titles, song_artists)),
              columns=['song_titles','song_artists'])


In [22]:
# PULL ALL AUDIO FEATURES FOR THE SONGS FROM MY PLAYLIST
#errors = []
#all_dicts = []

#for i in range(len(all_tracks)):
#  uri = all_tracks[i]["track"]["uri"]
#  try:  
#    all_dicts.append(list(sp.audio_features(uri)[0].values()))
#  except:
#    errors.append(i)
#    continue


# PUT AUDIO FEATURES INTO A PANDAS DATAFRAME
#df1 = pd.DataFrame(all_dicts, columns=['danceability', 'energy', 'key', 'loudness', 'mode', 
#                                      'speechiness','acousticness','instrumentalness','liveness','valence',
#                                       'tempo','type','id','uri','track_href',
#                                       'analysis_url','duration_ms','time_signature'])
# DROP THE SONGS FOR WHICH NO AUDIO FEATURES WERE FOUND 
# FROM THE DATA SET 
# new_artlist = spotify_data.drop(spotify_data.index[errors])
# new_artlist.reset_index(inplace=True)

# MERGE SONG TITLES WITH AUDIO FEATURES
# final_fuck = pd.concat([new_artlist, df1], axis = 1)

# STORE DATA INTO CSV TO SAVE LOADING TIME
# final_fuck.to_csv(r"/content/drive/MyDrive/playlist_final.csv")

# LOAD SONGS (INCL. AUDIO FEATURES) FROM CSV FILE
#longlist spotify
long_list = pd.read_csv("/content/drive/MyDrive/playlist_final.csv")

In [23]:
#List of all billboard and spotify together
spotify_billboard = pd.concat([final100, long_list], axis = 0)
spotify_billboard.reset_index(inplace=True)

In [24]:
#list with only numerical values
spotify_billboard_short = spotify_billboard.drop(['level_0','song_titles','song_artists', 'type','id','uri','track_href','analysis_url'], axis=1)
#spotify_billboard_short.reset_index(inplace= True)
spotify_billboard_short.drop(["index","Unnamed: 0"], axis=1, inplace=True)

In [36]:

def spotify_en(dfshort, dflong):
  #User Input clustering
  userinput = re.sub("^\s+|\s+$", "", input(str('Write down you favorite song: ')))
  #y= re.sub("\s+", " ", x)
  #z = y.lower()
  song = sp.search(q=userinput, limit=1) 
  print(song)
  song["tracks"]["items"][0]["uri"]
  sp.audio_features(song["tracks"]["items"][0]["uri"])[0]
  input_id = sp.audio_features(song["tracks"]["items"][0]["uri"])[0]['id']

  my_dict = sp.audio_features(song["tracks"]["items"][0]["uri"])[0]
  my_dict_new = { key: [my_dict[key]] for key in list(my_dict.keys()) }
  my_dict_new['name'] = [song["tracks"]["items"][0]['name']]
  #print(my_dict_new)

  example = pd.DataFrame(my_dict_new)
  #schritt 1
  example_1 = example.drop(['type','id','uri','track_href','analysis_url', 'name'], axis=1)

  new = pd.concat([dfshort, example_1], axis = 0, sort = False) #putting the new song below the original list

  #scaling the whole list
  scaler = StandardScaler()
  scaler.fit(new)
  new_scaled = scaler.transform(new)
  new_df = pd.DataFrame(new_scaled, columns = dfshort.columns)
  # scaler.transform(user_song)
  #display(new_df)
  #print()

    #clustering everything
  kmeans = KMeans(n_clusters=33, random_state=1234)
  kmeans.fit(new_df.iloc[:,:-1])
  #Clusteraufteilung anschauen
  clusters = kmeans.predict(new_df.iloc[:,:-1])
  pd.Series(clusters).value_counts().sort_index()
  #Final clustering
  all_scaled_df = pd.DataFrame(new_df)
  all_scaled_df["cluster"] = clusters

  if input_id in list(dflong[:100]['id']): #checking if the new list is part of the hot 100
    cluster_general = all_scaled_df[all_scaled_df['cluster'] == (list(all_scaled_df["cluster"])[-1])] #all the songs of the list which have the same cluster
    double_index = dflong[:100]["id"][dflong["id"] == input_id].index[0] #index of the input id in the top100 list
    sample_ids = list(cluster_general[cluster_general.index < 100].index)#list of all ids with the same cluster as the input id which are in top 100
    sample_ids.remove(double_index)#removing the input id

    #print(sample_ids)
    suggestion_idx = random.choice(sample_ids)#gives a random choice of the sample ids
    #print(suggestion_idx)#print the index of the suggested id
    #print(dflong.at[suggestion_idx, "song_titles"] + ", by: " + dflong.at[suggestion_idx, "song_artists"])#print the song title and artist of the new suggested song

  else:
    print("This song is not in the top 100 but I've got another one for you")
    cluster_general = all_scaled_df[all_scaled_df['cluster'] == (list(all_scaled_df["cluster"])[-1])]
    sample_ids = list(cluster_general[cluster_general.index >= 100].index)#all titles which are in the general playlist
    #print(cluster_general)
    try:
      double_index = dflong[100:]["id"][dflong["id"] == input_id].index[0]#check if the song is in the long playlist
      sample_ids.remove(double_index)#remove the ones which are the same 
    except:
      pass

    #print(sample_ids)
    suggestion_idx = random.choice(sample_ids)
    #print(suggestion_idx)
    #print(dflong.at[suggestion_idx, "song_titles"] + ", by: " + dflong.at[suggestion_idx, "song_artists"])

  return (dflong.at[suggestion_idx, "song_titles"] + ", by: " + dflong.at[suggestion_idx, "song_artists"])

spotify_en(spotify_billboard_short, spotify_billboard)
  

Write down you favorite song: mamma mia
{'tracks': {'href': 'https://api.spotify.com/v1/search?query=mamma+mia&type=track&offset=0&limit=1', 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LcJLqbBmaGUft1e9Mm8HV'}, 'href': 'https://api.spotify.com/v1/artists/0LcJLqbBmaGUft1e9Mm8HV', 'id': '0LcJLqbBmaGUft1e9Mm8HV', 'name': 'ABBA', 'type': 'artist', 'uri': 'spotify:artist:0LcJLqbBmaGUft1e9Mm8HV'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR

'Closer, by: Tegan and Sara'

In [ ]:
#try if cluster works the same
#spotify_billboard_short
#scaler = StandardScaler()
#scaler.fit(new)
#spotify_billboard_short_scaled = scaler.transform(spotify_billboard_short)
#spotify_billboard_short_df = pd.DataFrame(spotify_billboard_short_scaled, columns = spotify_billboard_short.columns)
# scaler.transform(user_song)

In [40]:
#filtering all the title with the same cluster as the cluster of the inserted song
#all_scaled_df[all_scaled_df['cluster'] == int(list(all_scaled_df["cluster"])[-1])]

In [41]:
#filtering all the title with the same cluster as the cluster of the inserted song, as list
#inter_df = all_scaled_df[all_scaled_df['cluster'] == (list(all_scaled_df["cluster"])[-1])]
#list(inter_df[inter_df.index < 100].index)

In [ ]:
#getting the optimal k elbow method
K = range(2, 60)
inertia = []
for k in K:
    #print("Training a K-Means model with {} neighbours! ".format(k))
    #print()
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(all_scaled_df)
    inertia.append(kmeans.inertia_)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#plt.figure(figsize=(16,8))
#plt.plot(K, inertia, 'bx-')
#plt.xlabel('k')
#plt.ylabel('inertia')
#plt.xticks(np.arange(min(K), max(K)+1, 1.0))
#plt.title('Elbow Method showing the optimal k')
#Getting the optimal k , silhouette mthod
import pickle
K = range(2, 60)
silhouette = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(all_scaled_df)
    filename = "/content/drive/MyDrive/kmeans1_" + str(k) + ".pickle"
    with open(filename, "wb") as f:
        pickle.dump(kmeans,f)
    silhouette.append(silhouette_score(all_scaled_df, kmeans.predict(Four_scaled_df)))

#plt.figure(figsize=(16,8))
#plt.plot(K, silhouette, 'bx-')
#plt.xlabel('k')
#plt.ylabel('silhouette score')
#plt.xticks(np.arange(min(K), max(K)+1, 1.0))
#plt.title('Silhouette Method showing the optimal k')